https://github.com/iolucas/dlnd-projects/blob/master/image-classification/dlnd_image_classification.ipynb


In [1]:
import math
import tensorflow as tf
print("TensorFlow V{}".format(tf.__version__))
import numpy as np
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
%matplotlib inline
import os
from collections import Counter

from sklearn.model_selection import train_test_split

from data_manager import load_dataset

import json

import psycopg2

import random

import pickle

TensorFlow V1.9.0


In [2]:
class StrokesDataset:
    def __init__(self, db_addr="localhost"):
        #Connect to db
        self.conn = psycopg2.connect("dbname='postgres' user='postgres' host='{}' password='lucas'".format(db_addr))
        self.cur = self.conn.cursor()
        
        #Retrieve and cache ids list to use to query data later
        self.cur.execute("SELECT id FROM samples LIMIT 220000")
        id_list = self.cur.fetchall()
        self.id_cache = [id_value[0] for id_value in id_list]
        
        #Lets save key_to_ind to ensure the labels will be in the same order all the time
        #Try to load key_to_ind, if not succeed, get it and save it
        try:
            key_list = pickle.load(open("key_list.pickle", "rb"))
            #self.key_to_ind = pickle.load(open("key_to_ind.pickle", "rb"))
            print("Loaded key_list file.")
        except:
            #Retrieve and cache labels
            self.cur.execute("SELECT DISTINCT key FROM samples LIMIT 300000")
            key_list = [key_value[0] for key_value in self.cur.fetchall()]
            pickle.dump(key_list, open("key_list.pickle", "wb"), protocol=pickle.HIGHEST_PROTOCOL)
            print("Generated new key_list file.")
            
        self.key_to_ind = dict([(key, ind) for ind, key in enumerate(key_list)])
        self.ind_to_key = key_list
            
            
        
        
    def get_train_valid_test_ids(self):
        train_ids, valid_ids = train_test_split(self.id_cache, test_size=0.4)
        valid_ids, test_ids = train_test_split(valid_ids, test_size=0.5)
        return tuple(train_ids), tuple(valid_ids), tuple(test_ids)
    
    def DEPRECTED_get_batches(self, ids, batch_size):
        assert batch_size < 5000
        
        self.cur.execute("SELECT key, strokes FROM samples WHERE id in {}".format(ids))
       
        while True:
            data_batch = self.cur.fetchmany(batch_size)
            if len(data_batch) == 0:
                break
                
            ziped = list(zip(*data_batch)) #Use zip to separate labels from datapoints
            
            yield ziped[1], ziped[0]
            
    def get_batches(self, ids, batch_size):
        assert batch_size < 5000
        
        for i in range(0,len(ids), batch_size):        
            self.cur.execute("SELECT key, strokes FROM samples WHERE id in {}".format(ids[i:i+batch_size]))
       
            data_batch = self.cur.fetchall()
                
            ziped = list(zip(*data_batch)) #Use zip to separate labels from datapoints
            
            yield ziped[1], ziped[0]   

In [3]:
def draw_image(strokes_list):
    im = Image.new(mode="1", size=(500,500))

    draw = ImageDraw.Draw(im)
    #draw.moveTo(strokes[0][:2], fill=128)
    #print(strokes)
    #draw.line((0, 0) + im.size, fill=128)
    #draw.line((0, im.size[1], im.size[0], 0), fill=128)
    for strokes in strokes_list:
        draw.line(strokes, fill=128, width=5)
    
    
    #for stroke in strokes[0:]:
        #print(stroke.tolist())
        #draw.line(stroke, fill=128, width=10)

    plt.imshow(im)

# write to stdout
#im.save(sys.stdout, "PNG")

In [4]:
def gen_tuples(strokes_list):
    strokes_group = list()
    for strokes in strokes_list:
        strokes_tuples = list()
        for x,y,t in strokes:
            strokes_tuples.append((x,y))
        strokes_group.append(strokes_tuples)
    return strokes_group

In [5]:
def gen_random_tuple(size):
    assert size < 100000
    rand_list = list()
    while len(rand_list) < size:
        rand_value = random.randint(1,50000)
        if rand_value not in rand_list:
            rand_list.append(rand_value)
        
    return tuple(rand_list)

In [6]:
ds = StrokesDataset("localhost")

Loaded key_list file.


In [7]:
ds.key_to_ind[ds.ind_to_key[1097]]

1097

In [8]:
import json

symbols = json.loads(open("../../dataset/detexify data/symbols.json").read())

$$ \sum $$

In [17]:
#gen_tuples(test_batch[0][0])

In [12]:
for i, b in enumerate(test_batch[1]):
    if 'sum' in b:
        draw_image(gen_tuples(test_batch[0][i]))
        plt.show()

NameError: name 'test_batch' is not defined

In [13]:
trainset, validset, testset = ds.get_train_valid_test_ids()

In [22]:
import pickle
dataset_tuples = list()

In [24]:
%time test_batch = next(ds.get_batches(trainset,1000))
for i in range(1000):
    #draw_image(gen_tuples(test_batch[0][i]))
    dataset_tuples.append(gen_tuples(test_batch[0][i]))
    #print(test_batch[1][59])
    #plt.show()

Wall time: 320 ms


In [25]:
pickle.dump(dataset_tuples, open( "save.p", "wb" ) )

In [11]:
def transform_data(strokes_list):
    """Convert strokes to match (deltaX, deltaY, is_drawing, is_moving, is_finished)"""
    
    onehots = list() #Gen pen moving onehots
    flatten_strokes = [[0,0,0]] #Reshape strokes lists
    
    for strokes in strokes_list:
        strokes = sorted(strokes, key=lambda a: a.__getitem__(2), reverse=False) #ensure strokes are in order
        flatten_strokes += strokes
        strokes_onehots = [[0,1,0]] + [[1,0,0]]*(len(strokes)-1)
        onehots += strokes_onehots
       
    #Set end flag at the last coordinate
    onehots[-1] = [0,0,1]
        
    flatten_strokes = np.array(flatten_strokes)
    delta_strokes = flatten_strokes[1:,:2] - flatten_strokes[:-1,:2] #Get delta values (:2 removes the timing data)
    
    np_batch = np.concatenate((delta_strokes, onehots), axis=1)
    
    return np_batch.astype(int)
    
%time transform_data(test_batch[0][600])
#draw_image(gen_tuples(test_batch[0][600]))

Wall time: 0 ns


array([[ 94, 139,   0,   1,   0],
       [  1,   0,   1,   0,   0],
       [  7,   0,   1,   0,   0],
       ...,
       [ -2,   2,   1,   0,   0],
       [  0,   1,   1,   0,   0],
       [  0,   1,   0,   0,   1]])

https://www.tensorflow.org/tutorials/recurrent_quickdraw<br>
https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell<br>
https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper<br>
https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell<br>
https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/stack_bidirectional_dynamic_rnn<br>
https://github.com/tensorflow/models/blob/master/tutorials/rnn/quickdraw/train_model.py<br>
https://github.com/udacity/deep-learning/blob/master/intro-to-rnns/Anna_KaRNNa_Solution.ipynb<br>
https://github.com/tensorflow/magenta/tree/master/magenta/models/sketch_rnn<br>

## Create Model

In [37]:
class RNNModel():
    def __init__(self, n_classes, n_lstm_layers=2, n_lstm_nodes=128, grad_clip=9.0, learning_rate=0.01, dropout=0.0, cudnn=False):      
        
        with tf.variable_scope("rnn_model", reuse=tf.AUTO_REUSE):
            
            #Placeholders
            inputs = tf.placeholder(tf.float32, shape=[None, None, 5], name="inputs")
            #keep_prob = tf.placeholder(tf.float32, name="keep_prob")
            targets = tf.placeholder(tf.int32, shape=[None], name="targets")
            targets_onehot = tf.one_hot(targets, n_classes)
            
            
            if cudnn:
                #Cudnn LSTM
                #CudnnLSTM is time-major, so we need to transpose
                inputs_t = tf.transpose(inputs, [1, 0, 2])

                lstm = tf.contrib.cudnn_rnn.CudnnLSTM(
                    num_layers=n_lstm_layers,
                    num_units=n_lstm_nodes,
                    dropout=dropout,
                    direction="bidirectional")

                lstm_outputs_t, _ = lstm(inputs_t)

                # Convert back from time-major outputs to batch-major outputs.
                lstm_outputs = tf.transpose(lstm_outputs_t, [1, 0, 2])
            else:
                
                def create_rnn_cell(lstm_size, keep_prob):
                    lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
                    cell = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob=keep_prob)
                    return cell
                
                fw_cells = [create_rnn_cell(n_lstm_nodes, 1 - dropout) for _ in range(n_lstm_layers)] #Forwards cells
                bw_cells = [create_rnn_cell(n_lstm_nodes, 1 - dropout) for _ in range(n_lstm_layers)] #Backwards cells
                
                lstm_outputs, _, _ = tf.contrib.rnn.stack_bidirectional_dynamic_rnn(
                    fw_cells,
                    bw_cells,
                    inputs,
                    dtype=tf.float32
                )

        
        
            #Outputs
            logits = tf.layers.dense(lstm_outputs[:,-1,:], n_classes)

            outputs = tf.nn.softmax(logits)

            loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=targets_onehot, logits=logits)
            loss = tf.reduce_mean(loss)

            #Create optimizers
            # Optimizer for training, using gradient clipping to control exploding gradients
            tvars = tf.trainable_variables()
            grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
            train_op = tf.train.AdamOptimizer(learning_rate)
            optimizer = train_op.apply_gradients(zip(grads, tvars))

            # Accuracy
            correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(targets_onehot, 1))
            accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')
            
            #Get references
            self.accuracy = accuracy
            self.inputs = inputs
            self.targets = targets
            self.loss = loss
            self.optimizer = optimizer
            self.outputs = outputs
            

In [38]:
#tf.reset_default_graph()
#RNNModel(10)
check whether beta values of training are being create twice
go aws fast!

SyntaxError: invalid syntax (<ipython-input-38-119837663943>, line 3)

In [19]:
#dir(tf.GraphKeys)

In [20]:
#tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)

In [21]:
#tf.global_variables()

In [22]:
def _add_cudnn_rnn_layers(convolved, num_nodes, num_layers, keep_prob):
    """Adds CUDNN LSTM layers."""
    # Convolutions output [B, L, Ch], while CudnnLSTM is time-major.
    
    convolved = tf.transpose(convolved, [1, 0, 2])
    lstm = tf.contrib.cudnn_rnn.CudnnLSTM(
        num_layers=num_layers,
        num_units=num_nodes,
        dropout=keep_prob,
        direction="bidirectional")
    outputs, _ = lstm(convolved)
    # Convert back from time-major outputs to batch-major outputs.
    outputs = tf.transpose(outputs, [1, 0, 2])
    return outputs

In [40]:
tf.reset_default_graph()

#Create network
#lstm_size = 128
#num_layers = 2
#grad_clip = 9.0
#learning_rate = 0.01
n_classes = len(ds.key_to_ind)

#tf.reset_default_graph()
train_rnn_model = RNNModel(n_classes=n_classes, dropout=0.0)
infer_rnn_model = RNNModel(n_classes=n_classes, dropout=0.0)

#inputs = tf.placeholder(tf.float32, shape=[None, None, 5], name="inputs")
#keep_prob = tf.placeholder(tf.float32, name="keep_prob")

#targets = tf.placeholder(tf.int32, shape=[None], name="targets")

#targets_onehot = tf.one_hot(targets, n_classes)


#rnn_outputs = _add_cudnn_rnn_layers(inputs, lstm_size, num_layers, keep_prob)


#fw_cells = [create_rnn_cell(lstm_size, keep_prob)] #Forwards cells
#bw_cells = [create_rnn_cell(lstm_size, keep_prob)] #Backwards cells

#rnn_outputs, output_state_fw, output_state_bw = tf.contrib.rnn.stack_bidirectional_dynamic_rnn(
    #fw_cells, bw_cells, inputs, dtype=tf.float32)

#Use [:,-1,:] to keep only the (batches, last outputs, classes)
#logits = tf.layers.dense(rnn_outputs[:,-1,:], n_classes)

#outputs = tf.nn.softmax(logits)

#loss = tf.nn.softmax_cross_entropy_with_logits(labels=targets_onehot, logits=logits)
#loss = tf.reduce_mean(loss)

#Create optimizers
#optimizer = tf.train.AdamOptimizer(0.01).minimize(cost)

# Optimizer for training, using gradient clipping to control exploding gradients
#tvars = tf.trainable_variables()
#grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
#train_op = tf.train.AdamOptimizer(learning_rate)
#optimizer = train_op.apply_gradients(zip(grads, tvars))

# Accuracy
#correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(targets_onehot, 1))
#accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')

In [41]:
def get_batches(dataset, batch_size):
    for x_batch, y_batch in ds.get_batches(dataset, batch_size):
        #Transform X data to get delta cordinates, 
        trans_x = list(map(transform_data, x_batch))
        
        #make them the same size of the max size        
        max_size = max(map(len, trans_x))
        x_data_buffer = np.array([0,0,0,0,1]) * np.ones((batch_size, max_size, 5)).astype(int)
        for i, x_transf_data in enumerate(trans_x):
            x_data_buffer[i][:len(x_transf_data)] = x_transf_data
        
        #transform y data to get indexes for each label
        yield x_data_buffer, list(map(ds.key_to_ind.get, y_batch))
 
#Test
#%time testdata = next(get_batches(trainset, 5))
#print(testdata[0].shape)

In [42]:
#%time draw_image(gen_tuples(next(test_batches)[0][8]))

In [43]:
len(trainset)

126272

In [55]:
126272/256

493.25

In [60]:
math.floor(len(validset)/256)*256

41984

In [ ]:
epochs = 1

train_batch_size = 256
valid_batch_size = 256

max_trainset_len = math.floor(len(trainset)/train_batch_size) * train_batch_size
max_validset_len = math.floor(len(validset)/valid_batch_size) * valid_batch_size

sess = tf.Session()

ds = StrokesDataset("189.62.80.32")

sess.run(tf.global_variables_initializer())

loss_buffer = list()

loss = train_rnn_model.loss
optimizer = train_rnn_model.optimizer
t_inputs = train_rnn_model.inputs
t_targets = train_rnn_model.targets

accuracy = infer_rnn_model.accuracy
a_inputs = infer_rnn_model.inputs
a_targets = infer_rnn_model.targets

for e in range(epochs):
    
    batch_iter = 0
    
    for X_batch, Y_batch in get_batches(trainset[:max_trainset_len], train_batch_size):
        batch_iter += 1
            
        loss_value, _ = sess.run([loss, optimizer], feed_dict={
            t_inputs: X_batch,
            t_targets: Y_batch
        })
        
        loss_buffer.append(loss_value)        
        print(loss_value)
        
        
        if batch_iter % 10 == 0:
        
            print("Working on accuracy...")
            #Check accuracy
            acc_list = list()
            for X_valid, Y_valid in get_batches(validset[:max_validset_len], valid_batch_size):

                print("Calculating acc...")

                accuracy_value = sess.run(accuracy, feed_dict={
                    a_inputs: X_valid,
                    a_targets: Y_valid
                })

                acc_list.append(accuracy_value)

            valid_accuracy = sum(acc_list) / len(acc_list)     
            print(valid_accuracy)

    #acc_value = sess.run([accuracy], feed_dict={
        #inputs: X_valid,
        #targets: y_valid
    #})
    
    #print(e, loss_value, acc_value)

Loaded key_list file.
7.001681
6.8799314
6.449214
6.635103
6.313327
6.273525
6.2732987
6.2320237
6.227172
6.2402124
Working on accuracy...
Calculating acc...
Calculating acc...
Calculating acc...
Calculating acc...
Calculating acc...
Calculating acc...
Calculating acc...
Calculating acc...
Calculating acc...
Calculating acc...
Calculating acc...
Calculating acc...
Calculating acc...
Calculating acc...
Calculating acc...
Calculating acc...
Calculating acc...
Calculating acc...
Calculating acc...
Calculating acc...
Calculating acc...
Calculating acc...
Calculating acc...
Calculating acc...
Calculating acc...
Calculating acc...
Calculating acc...
Calculating acc...
Calculating acc...
Calculating acc...
Calculating acc...
Calculating acc...
